# 1 - Two-agent Chat

## Setup

Material de referencia: https://autogen-ai.github.io/autogen/docs/topics/llm_configuration/#adding-http-client-in-llm_config-for-proxy

In [58]:
#import libs
from autogen import ConversableAgent
import os
import pprint

## Definindo os agentes

In [85]:
student_agent = ConversableAgent(
    name="agente_estudante",
    system_message="Você é um estudante disposto a aprender.",
    llm_config={
        "model": "gemma2:2b",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
        "temperature":2
    },
)
teacher_agent = ConversableAgent(
    name="agente_professor",
    system_message="Você é um professor de matemática.",
    llm_config={
        "model": "llama3-70b-8192",
        "base_url": "https://api.groq.com/openai/v1",  
        "api_key": os.getenv("GROQ_API_KEY"),
        "temperature":1  
    },
)

## Colocando os agentes para conversar

In [86]:
chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="Me explique a formula de Bhaskara?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

agente_estudante (to agente_professor):

Me explique a formula de Bhaskara?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 09-24 17:44:08] {349} WARNING - Model llama3-70b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
agente_professor (to agente_estudante):

Uma das minhas fórmulas favoritas!

A fórmula de Bhaskara, também conhecida como fórmula quadrada, é uma fórmula matemática que fornece a solução para equações do segundo grau da forma ax^2 + bx + c = 0, onde a, b e c são constantes e x é a variável.

A fórmula de Bhaskara é:

x = (-b ± √(b^2 - 4ac)) / 2a

Simples, não é?

Vamos explicar cada parte da fórmula:

* x é a solução que estamos procurando;
* -b é o negativo do coeficiente do termo linear (o termo com x);
* ± é o símbolo de mais ou menos, indicando que existem dois possíveis v

## Visualizando alguns resultados

Voce pode visualizar:

1. Historico do Chat
2. Custo
3. Summario da conversa

In [82]:
pprint.pprint(chat_result.chat_history)

[{'content': 'Me explique a formula de Bhaskara?',
  'name': 'agente_estudante',
  'role': 'assistant'},
 {'content': 'Olá! 👋É um prazer estar com você em uma jornada pelos '
             'maravilhosos fundamentos matemáticas com o conceito do Teorema '
             'deBhaskara! 🔥 🤯 . 😄 A sua curiosidade é louca! 🤔\n'
             '\n'
             'Certo- que vamos entender suas notas de matemática! 😎  A fórmula '
             'dos Bhasksara se inicia de modo elegante e direto nos ajudará a '
             'resolver um assínftico! 😊 🤓\n'
             '\n'
             '\n'
             '#### **Conceitos Fundamentales:**\n'
             '\n'
             'As funções matemática da álgebra tem, de longe, grande impacto '
             'na complexidade em muitos aspectos do contexto. 📚\n'
             '\n'
             'Para entender melhor:  As equações de segundo gradou é bastante '
             'básica e fundamental para o entresse em geometrias ou álgebras '
             'complexão as m

In [83]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gemma2:2b': {'completion_tokens': 1585,
                                                    'cost': 0,
                                                    'prompt_tokens': 3376,
                                                    'total_tokens': 4961},
                                      'llama3-70b-8192': {'completion_tokens': 544,
                                                          'cost': 0,
                                                          'prompt_tokens': 1197,
                                                          'total_tokens': 1741},
                                      'total_cost': 0},
 'usage_including_cached_inference': {'gemma2:2b': {'completion_tokens': 1585,
                                                    'cost': 0,
                                                    'prompt_tokens': 3376,
                                                    'total_tokens': 4961},
                                      'llama3

In [84]:
pprint.pprint(chat_result.summary)

('1 - The formula for finding roots  for second-highest-power equations was '
 'covered, emphasizing:\n'
 '\n'
 '\n'
 '   * Its clarity (in simplified versions) as a key method of dealing with '
 'equations\n'
 '\n'
 '\n'
 '. We used this equation of the second of degree \n'
 '\n'
 '2. A summary of learning concepts in clear descriptions\n'
 '\n'
 '**For Exercises** You Should practice it:\n'
 '\n'
 '\n'
 '\n'
 '1. x + y =\n'
 '20\n'
 '\n'
 '3 -  If anyone has been taught about an object, to create his root of '
 'solutions or some of those skills that might look simple for me.. 😉! Do '
 'try!\n'
 '\n'
 '\n'
 '\n'
 '\n'
 ' .\n'
 '\n'
 '   \n'
 '\n'
 '\n')
